<a href="https://colab.research.google.com/github/versant2612/jnotebooks/blob/main/kgtk/03_kg_graph_embeddingsComplEx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kgtk==1.0.1

     |████████████████████████████████| 550 kB 11.7 MB/s 
     |████████████████████████████████| 88 kB 7.1 MB/s 
     |████████████████████████████████| 78 kB 5.2 MB/s 
     |████████████████████████████████| 407 kB 49.2 MB/s 
     |████████████████████████████████| 118 kB 68.2 MB/s 
     |████████████████████████████████| 57 kB 3.7 MB/s 
     |████████████████████████████████| 81 kB 7.4 MB/s 
     |████████████████████████████████| 203 kB 52.0 MB/s 
     |████████████████████████████████| 167 kB 71.2 MB/s 
     |████████████████████████████████| 99 kB 7.9 MB/s 
     |████████████████████████████████| 1.8 MB 51.5 MB/s 
     |████████████████████████████████| 130 kB 94.2 MB/s 
     |████████████████████████████████| 1.2 MB 50.4 MB/s 
     |████████████████████████████████| 21.5 MB 1.3 MB/s 
     |████████████████████████████████| 40 kB 4.9 MB/s 
     |████████████████████████████████| 10.1 MB 58.2 MB/s 
     |████████████████████████████████| 78 kB 6.7 MB/s 
     |█████████████████████

# KGTK graph-embeddings

`kgtk graph-embeddings` command takes as input a KGTK edge file and compute the embeddings of the graph nodes and relations.

Please refer to [graph-embeddings documentation](https://kgtk.readthedocs.io/en/latest/analysis/graph_embeddings/) for further details.

In [1]:
import os
from kgtk.configure_kgtk_notebooks import ConfigureKGTK
from kgtk.functions import kgtk, kypher
from gensim.models import KeyedVectors
import tempfile
import pandas as pd
import numpy as np
import h5py, torch
from torchbiggraph.model import ComplexDiagonalDynamicOperator, DotComparator, CosComparator
import json

In [2]:
# Parameters

# Folder on local machine where to create the output and temporary folders
input_path = None
output_path = "/tmp/projects"
project_name = "tutorial-graph-embeddings"
input_files_url="https://github.com/usc-isi-i2/kgtk-tutorial-files/raw/main/datasets/arnold-profiled"

#### Define a custom json files config to download the file `derived_P31x` as it is not part of default download list

In [3]:
extra_files_config = {
    "derived_P31x": "derived.P31x.tsv"
    }

open('/root/extra_files.json', 'w').write(json.dumps(extra_files_config))

36

In [4]:
files = [
    "all",
    "label",
    "alias",
    "description",
    "item",
    "qualifiers",
    "p31",
    "p279star",
    "derived_P31x"
]
ck = ConfigureKGTK(files, input_files_url=input_files_url)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name,
                  json_config_file = '/root/extra_files.json')

User home: /root
Current dir: /content
KGTK dir: /
Use-cases dir: //use-cases
--2021-11-22 01:37:53--  https://github.com/usc-isi-i2/kgtk-tutorial-files/raw/main/datasets/arnold-profiled/all.tsv.gz
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/usc-isi-i2/kgtk-notebooks/raw/main/datasets/arnold-profiled/all.tsv.gz [following]
--2021-11-22 01:37:53--  https://github.com/usc-isi-i2/kgtk-notebooks/raw/main/datasets/arnold-profiled/all.tsv.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/datasets/arnold-profiled/all.tsv.gz [following]
--2021-11-22 01:37:53--  https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/datasets/arnold-profiled/all.tsv.gz
Resolving raw.githubusercontent.com (raw.github

In [5]:
ck.print_env_variables()

KGTK_GRAPH_CACHE: /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db
KGTK_OPTION_DEBUG: false
KGTK_LABEL_FILE: /root/isi-kgtk-tutorial/input/labels.en.tsv.gz
TEMP: /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings
USE_CASES_DIR: //use-cases
GRAPH: /root/isi-kgtk-tutorial/input
OUT: /tmp/projects/tutorial-graph-embeddings
STORE: /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db
kypher: kgtk query --graph-cache /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db
EXAMPLES_DIR: //examples
kgtk: kgtk
all: /root/isi-kgtk-tutorial/input/all.tsv.gz
label: /root/isi-kgtk-tutorial/input/labels.en.tsv.gz
alias: /root/isi-kgtk-tutorial/input/aliases.en.tsv.gz
description: /root/isi-kgtk-tutorial/input/descriptions.en.tsv.gz
item: /root/isi-kgtk-tutorial/input/claims.wikibase-item.tsv.gz
qualifiers: /root/isi-kgtk-tutorial/input/qualifiers.tsv.gz
p31: /root/is

In [6]:
ck.load_files_into_cache()

kgtk query --graph-cache /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/wikidata.sqlite3.db -i "/root/isi-kgtk-tutorial/input/all.tsv.gz" --as all  -i "/root/isi-kgtk-tutorial/input/labels.en.tsv.gz" --as label  -i "/root/isi-kgtk-tutorial/input/aliases.en.tsv.gz" --as alias  -i "/root/isi-kgtk-tutorial/input/descriptions.en.tsv.gz" --as description  -i "/root/isi-kgtk-tutorial/input/claims.wikibase-item.tsv.gz" --as item  -i "/root/isi-kgtk-tutorial/input/qualifiers.tsv.gz" --as qualifiers  -i "/root/isi-kgtk-tutorial/input/derived.P31.tsv.gz" --as p31  -i "/root/isi-kgtk-tutorial/input/derived.P279star.tsv.gz" --as p279star  -i "/root/isi-kgtk-tutorial/input/derived.P31x.tsv" --as derived_P31x  --limit 3
node1	label	node2	id	node2;wikidatatype
P10	alias	'gif'@en	P10-alias-en-282226-0	
P10	alias	'animation'@en	P10-alias-en-2f86d8-0	
P10	alias	'media'@en	P10-alias-en-c1427e-0	


In [7]:
# dimension of the output ComplEx vector
vector_dimension = 30 

# output path for embeddings file in KGTK format 
vector_output_path = f"{os.environ['OUT']}/arnold.embeddings.augmented.{vector_dimension}.tsv" 

# output path for embeddings file in w2v format
vector_output_w2v_path = f"{os.environ['OUT']}/arnold.embeddings.augmented.{vector_dimension}.w2v.tsv"

# output path for embeddings file in w2v format - 2
vector_output_w2v_path2 = f"{os.environ['OUT']}/arnold.embeddings.augmented.{vector_dimension}.w2v.2.tsv"

os.environ['VECTOR_DIMENSION'] = str(vector_dimension)

## Compute ComplEx Graph Embeddings

In this notebook we will compute graph embeddings using `kgtk graph-embeddings` command for the `arnold` subgraph and demonstrate a few applications.

First step is to augment the `claims.wikibase-item.tsv.gz` file with `derived.P31x.tsv` file which contains occupations for humans as `instance of (P31)`

- `claims.wikibase-item.tsv.gz`: KGTK claims file non literal edges only
- `derived.P31x.tsv`: file with additional P31x links, adding occupation as `instance of` (computed)

In [8]:
!kgtk cat -i $item \
-i $GRAPH/derived.P31x.tsv \
-o $GRAPH/claims.wikibase-item.augmented.tsv.gz

### Run `kgtk graph-embeddings`

The `kgtk graph-embeddings` command takes as input a KGTK edge file and computes graph embeddings of user specified type, producing vectors of user specified dimensions.

The following parameters are used in this instance:

- `-op ComplEx`: compute ComplEx graph embeddings
- `--dimension 30`: desired dimension of the vectors
- `-ot kgtk`: output format - kgtk
- `--retain_temporary_data True`: retain the byproduct files, which we will use in subsequent steps
- `-T <folder path>`: temporary folder where the temporary files will be stored
- `-i <file>`: input file
- `-o <file>`: output file
- `--log <file>`: log file

**NOTE**: This cell will take ~15 minutes to run on the default Google Colab VM.

In [9]:
kgtk(f""" graph-embeddings
            -op ComplEx \
            --dimension $VECTOR_DIMENSION \
            -ot kgtk \
            --retain_temporary_data True \
            -T $TEMP \
            -i $GRAPH/claims.wikibase-item.augmented.tsv.gz \
            -o {vector_output_path} \
            --log $TEMP/ge.log.txt
    """)

In Processing, Please go to /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/ge.log.txt to check details
Processed Finished.



#### Take a peek at the embeddings file.

In [10]:
kgtk(f"""head -i {vector_output_path}""")

,node1,label,node2
0,Q881146,graph_embeddings,"-0.163394749,-0.141179964,0.661213279,-0.48888..."
1,Q104028,graph_embeddings,"0.559447527,-0.823413849,-0.645695925,-0.10887..."
2,Q7836079,graph_embeddings,"0.161949247,-0.031712286,0.340728611,-0.178901..."
3,Q52488603,graph_embeddings,"0.215596169,-0.038513701,-0.167117938,0.037203..."
4,Q2300373,graph_embeddings,"0.628062963,-0.202323362,-0.793009102,-0.16938..."
5,Q114,graph_embeddings,"0.653213441,0.270816833,-0.569554985,-0.151231..."
6,Q1418116,graph_embeddings,"1.080384612,-0.408172369,-0.310347915,1.071385..."
7,Q484367,graph_embeddings,"0.149368614,0.270516217,0.065050580,0.48716059..."
8,Q113864,graph_embeddings,"0.068800524,-0.712486744,0.435138643,-0.328124..."
9,Q1261096,graph_embeddings,"0.458130002,0.078747727,-0.079980500,0.3641195..."


In [11]:
kgtk(f""" graph-embeddings
            -op ComplEx \
            --dimension $VECTOR_DIMENSION \
            -ot w2v \
            --retain_temporary_data True \
            -T $TEMP \
            -i $GRAPH/claims.wikibase-item.augmented.tsv.gz \
            -o {vector_output_w2v_path2} \
            --log $TEMP/ge.log.txt
    """)

kgtk(f"""head -i {vector_output_w2v_path2}""")

In Processing, Please go to /tmp/projects/tutorial-graph-embeddings/temp.tutorial-graph-embeddings/ge.log.txt to check details
Processed Finished.

In input header '66014 30': 
Invalid Quantity



,66014 30
0,Q1435177 0.364979625 1.031170368 -0.832451642 ...
1,Q187334 -0.236916274 1.256894112 -0.895162702 ...
2,Q763978 0.354073673 0.981169879 -0.415806025 -...
3,Q95068 -0.100245282 0.930549800 -0.075429201 -...
4,Q236788 -0.080324180 0.934013128 -0.619165003 ...
5,Q21855572 -0.300672024 1.056717515 -0.32355687...
6,Q3230368 0.443160862 0.782289386 -0.764819682 ...
7,Q846592 0.549145520 0.673063874 -0.742440224 -...
8,Q165763 0.067976587 1.074807644 -0.843032539 0...
9,Q76853179 0.027138174 0.841985404 -0.543409586...


### The output is in `kgtk` format. Convert it to `word2vec` format for `gensim` similarity computation


For reference: 
- [gensim](https://radimrehurek.com/gensim/)
- [word2vec](https://en.wikipedia.org/wiki/Word2vec)

O header do w2v precisa conter o número de nós e o numero de dimensões. O vetor é separado por espaços. 

In [12]:
def convert_kgtk_to_w2v(input_path, output_path):
    """
    Convert a KGTK file (node1/label/node2) that contains embeddings to the w2v format
    """
    vector_count = 0

    # Read the file once to count the lines as we need to put them at the top of the w2v file
    with open(input_path, "r") as kgtk_file:
        next(kgtk_file)
        for line in kgtk_file:
            vector_count += 1
        kgtk_file.close()

    with open(output_path, "w") as w2v_file:
        w2v_file.write("{} {}\n".format(vector_count, vector_dimension))
        with open(input_path, "r") as kgtk_file:
            next(kgtk_file)
            for line in kgtk_file:
                items = line.split("\t")
                qnode = items[0]
                vector = items[2].replace(",", " ")
                w2v_file.write(qnode + " " + vector)
            kgtk_file.close()
        w2v_file.close()

In [13]:
convert_kgtk_to_w2v(f"{vector_output_path}", f"{vector_output_w2v_path}")

### Load the vectors into `gensim`

To find similar vectors based on cosine similarity

In [14]:
# ge_vectors = KeyedVectors.load_word2vec_format(f"{vector_output_w2v_path}", binary=False)
ge_vectors = KeyedVectors.load_word2vec_format(f"{vector_output_w2v_path2}", binary=False)

Define a function to compute the `topn` similar vectors, and get the labels and descriptions of the matching Qnodes.

Dado um vetor de entrada contendo o ID e os embeddings a função abaixo do pacote gensim retorna para um ID (positive) os topn IDs mais semelhantes. 

vectors.most_similar(positive=positive, topn=topn)

In [15]:
def kgtk_most_similar(
    vectors,
    positive,
    relation_label="similarity_score",
    add_label_description=True,
    output_path=None,
    topn=25,
):
    """
    find topn similar Qnodes, add label and decription for the Qnodes
    
    :param vectors: vector space loaded into gensim KeyedVectors model
    :param positive: vector(s) or Qnode(s) to find similar entities for
    :param relation_label: name of the property to be used for the output file
    :param add_label_description: boolean parameter to add label and description for matched entities
    :param output_path: path to store the output file
    :param topn: desirednumber of similar entities
    """
    result = []
    if add_label_description:
        fp = tempfile.NamedTemporaryFile(
            mode="w", suffix=".tsv", delete=False, encoding="utf-8"
        )
        fp.write("node1\tlabel\tnode2\n")
        for (qnode, similarity) in vectors.most_similar(positive=positive, topn=topn):
            fp.write("{}\t{}\t{}\n".format(qnode, relation_label, similarity))
        filename = fp.name
        fp.close()

        os.environ["_temp_file"] = filename

        result = !$kypher -i label -i description -i "$_temp_file" --as sim \
--match 'sim: (n1)-[]->(similarity), label: (n1)-[]->(lab), description: (n1)-[]->(des)' \
--return 'distinct n1 as node1, similarity as node2, "similarity" as label, lab as `node1;label`, des as `node1;description`' \
--order-by 'cast(similarity, float) desc' 
        
        os.remove(filename)
        
    else:
        result.append("node1\tlabel\tnode2\n")
        for (qnode, similarity) in vectors.most_similar(positive=positive, topn=topn):
            result.append("{}\t{}\t{}\n".format(qnode, relation_label, similarity))

    if output_path:
        handle = open(output_path, "w")
        for line in result:
            handle.write(line)
            handle.write("\n")
        handle.close()
    else:
        columns = result[0].split("\t")
        data = []
        for line in result[1:]:
            data.append(line.split("\t"))
        return pd.DataFrame(data, columns=columns)

### Link Prediction

The following code reads the vectors for Qnodes as `head` and Properties as `relation`.

The files used in the code are produced by `kgtk graph-embeddings` code as a byproduct, in the folder specified by the `-T` option

In [16]:
relation_names_list = json.load(open(f"{os.environ['TEMP']}/output/dynamic_rel_names.json"))
entity_names_list = json.load(open(f"{os.environ['TEMP']}/output/entity_names_all_0.json"))
prop_count = len(relation_names_list)

# operators
operator_lhs = ComplexDiagonalDynamicOperator(vector_dimension, prop_count)
operator_rhs = ComplexDiagonalDynamicOperator(vector_dimension, prop_count)
comparator = DotComparator()
cos_comparator = CosComparator()
with h5py.File(f"{os.environ['TEMP']}/output/model/model.v100.h5", "r") as hf:
    operator_state_dict_lhs = {
        "real": torch.from_numpy(hf["model/relations/0/operator/lhs/real"][...]),
        "imag": torch.from_numpy(hf["model/relations/0/operator/lhs/imag"][...]),
    }
    operator_state_dict_rhs = {
        "real": torch.from_numpy(hf["model/relations/0/operator/rhs/real"][...]),
        "imag": torch.from_numpy(hf["model/relations/0/operator/rhs/imag"][...]),
    }
    
operator_lhs.load_state_dict(operator_state_dict_lhs)
operator_rhs.load_state_dict(operator_state_dict_rhs)

# Load the embeddings
with h5py.File(f"{os.environ['TEMP']}/output/model/embeddings_all_0.v100.h5", "r") as hf:
    arnold_embedding = torch.from_numpy(hf["embeddings"][...])


entity_to_index = {}
for i, entity in enumerate(entity_names_list):
    entity_to_index[entity] = i
    

rel_index = {}
for i, rel in enumerate(relation_names_list):
    rel_index[rel] = i

The following function takes as input a `Qnode` and a `Property`, and outputs a vector which should be similar to the value of the relation.

For example, Qnode: `Q37079` = Tom Cruise, Property: `P166` = awards received and output a vector similar to awards. We will see this equation in action in the subsequent examples.

head + relation ~= tail *(h,r,?)*

In [17]:
def get_embed(head, relation=None):
    ''' This function generate the embeddings for the tail entities:
            Head entities: Obtained from the model
            Head + relation: Obtained using torch
        :param head: subject Qnode
        :param relation: optional property
    '''
    if relation is None:
        return arnold_embedding[entity_to_index[head], :].detach().numpy()
    return  operator_lhs(
                arnold_embedding[entity_to_index[head], :].view(1, vector_dimension),
                torch.tensor([rel_index[relation]])
            ).detach().numpy()[0]

#### Get the vector for `Q37079` (Tom Cruise) + `P166` (award received), then find most similar entities

a função get_embed recebe o head e o relation e "soma" para achar o tail aproximado e retorna o vetor de embeddings desse tail. Depois esse vetor é usado como o referencial de entrada para a kgtk_most_similar (parametro positive) para obter os 10 (topn) mais similares.

In [18]:
_vector = get_embed('Q37079', 'P166')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

,node1,node2,label,node1;label,node1;description
0,Q3404523,0.13750946521759033,similarity,'David di Donatello for Best Foreign Actor'@en,'former annual Italian film award'@en
1,Q103916,0.11780834197998047,similarity,'Academy Award for Best Actor'@en,'award presented annually by the Academy of Mo...
2,Q908858,0.10972359776496887,similarity,'Donostia Award'@en,'award'@en
3,Q2969900,0.10315485298633575,similarity,"'Château Miraval, Correns-Var'@en",'French wine chateau'@en
4,Q3703451,0.09589187055826187,similarity,'David di Donatello Award for Best Supporting ...,'annual Italian film award'@en
5,Q922322,0.09531483054161072,similarity,'Critics\' Choice Movie Award for Best Actor'@en,'award given by the Broadcast Film Critics Ass...
6,Q654620,0.07543483376502991,similarity,'Screen Actors Guild Award for Outstanding Per...,'Award presented by the Screen Actors Guild'@en


#### Get the vector for `Q170564` (Terminator 2: Judgement Day) + `P161` (cast member), then find most similar entities

In [19]:
_vector = get_embed('Q170564', 'P161')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

,node1,node2,label,node1;label,node1;description
0,Q150482,0.17065021395683289,similarity,'J. K. Simmons'@en,'American actor'@en
1,Q381203,0.1604847013950348,similarity,'Ted Danson'@en,'American actor'@en
2,Q483771,0.1582128405570984,similarity,'Elijah Wood'@en,'American actor'@en
3,Q3110380,0.15295477211475372,similarity,'Gustaf Tenggren'@en,'Swedish illustrator (1896-1970)'@en
4,Q313522,0.13427628576755524,similarity,'Rip Torn'@en,'American actor (1931-2019)'@en
5,Q312077,0.13220152258872986,similarity,'Lance Henriksen'@en,'American actor'@en
6,Q16253434,0.1304423213005066,similarity,'Thomas Francis Murphy'@en,'American actor'@en
7,Q464320,0.12553006410598755,similarity,'Stephen Root'@en,'American actor and voice actor'@en
8,Q229137,0.12539978325366974,similarity,'Karen Allen'@en,'American actress'@en
9,Q898187,0.12394140660762787,similarity,'Brandon Stacy'@en,'American actor'@en


#### Get the vector for `Q104123` (Pulp Fiction) + `P161` (cast member), then find most similar entities

In [20]:
_vector = get_embed('Q104123', 'P161')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

,node1,node2,label,node1;label,node1;description
0,Q108283,0.13864898681640625,similarity,'Alan Arkin'@en,"'American actor, director, musician and singer..."
1,Q381203,0.13778412342071533,similarity,'Ted Danson'@en,'American actor'@en
2,Q103784,0.1334545910358429,similarity,'Donald Sutherland'@en,'Canadian actor'@en
3,Q174843,0.132558211684227,similarity,'Jeff Bridges'@en,"'American actor, singer-songwriter and produce..."
4,Q152208,0.12962603569030762,similarity,'Gisele Bündchen'@en,'Brazilian fashion model'@en
5,Q178552,0.1289222091436386,similarity,'Mickey Rourke'@en,"'American actor, screenwriter, and former boxe..."
6,Q102711,0.12851130962371826,similarity,'Dennis Hopper'@en,'American actor and filmmaker (1936–2010)'@en
7,Q165219,0.1238463968038559,similarity,'Robert Downey Jr.'@en,'American actor from New York'@en
8,Q192165,0.12233926355838776,similarity,'Danny Glover'@en,"'American actor, film director and political a..."
9,Q48337,0.11742600053548813,similarity,'Morgan Freeman'@en,"'American actor, film director, and narrator'@en"


#### Get the vector for `Q2685` (Arnold Schwarzenegger), then find most similar entities

a função get_embed recebe o ID do nó e retorna o vetor de embeddings desse ID. Depois esse vetor é usado como o referencial de entrada para a kgtk_most_similar (parametro positive) para obter os 10 (topn) mais similares.

In [21]:
_vector = get_embed('Q2685')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

,node1,node2,label,node1;label,node1;description
0,Q2685,0.9999999403953552,similarity,'Arnold Schwarzenegger'@en,"'Austrian-American actor, businessman, bodybui..."
1,Q243430,0.9023052453994751,similarity,'Terence Hill'@en,"'Italian actor, film director, screenwriter an..."
2,Q272972,0.8693974614143372,similarity,'Robin Wright'@en,'American actress'@en
3,Q110374,0.8674886226654053,similarity,'Bill Paxton'@en,"'American actor, director and film producer (1..."
4,Q320218,0.8585519194602966,similarity,'Carl Weathers'@en,'actor; Former American Football Player'@en
5,Q539718,0.856234610080719,similarity,'Joel Silver'@en,'American businessman and film producer'@en
6,Q311314,0.8558782339096069,similarity,'Sam Elliott'@en,'American actor'@en
7,Q94715,0.8434388041496277,similarity,'Dagmar Koller'@en,'Austrian actor and singer'@en
8,Q1386741,0.8408252000808716,similarity,'Harve Bennett'@en,'American television and film producer (1930-2...
9,Q9960,0.8404729962348938,similarity,'Ronald Reagan'@en,'40th president of the United States and actor...


#### Get the vector for `Q103148` (Lahn River), then find most similar entities

In [22]:
_vector = get_embed('Q103148')
kgtk_most_similar(ge_vectors, positive=[_vector], topn=10)

,node1,node2,label,node1;label,node1;description
0,Q103148,1.0,similarity,'Lahn'@en,'right tributary of the Rhine River in Germany...
1,Q153945,0.9693168997764587,similarity,'Lippe'@en,'river in Germany'@en
2,Q26727445,0.9679651856422424,similarity,'Moersbach'@en,'river in Germany'@en
3,Q570612,0.966664731502533,similarity,'Acher'@en,'river in Germany'@en
4,Q701645,0.9660314917564392,similarity,'Düssel'@en,'river in Germany'@en
5,Q326366,0.9632155895233154,similarity,'Leimbach'@en,'river'@en
6,Q319649,0.9626142978668213,similarity,'Möhlin'@en,'river in Germany'@en
7,Q168696,0.9535875916481018,similarity,'Nahe'@en,'tributary of Rhine river'@en
8,Q563698,0.9509090185165405,similarity,'Lauter'@en,"'river in Germany and France, left tributary o..."
9,Q639374,0.9497984647750854,similarity,'Wupper'@en,'river in Germany'@en


## Prepare files for Google Projector

In this section, we will prepare `vectors` and `metadata` files for google projector.

We are focusing on the following types:

- `Q11424` (film)
- `Q33999` (actor)
- `Q4022` (river)
- `Q82955` (politician)

First step is to create a file with the following information ,

1. node1 :- Qnode
2. label :- name of the property
3. node2 :- embedding vector for node1
4. node1;label :- label for node1
5. type :- `instance of` for node1
6. type;label :- label for type

In [23]:
%%time
kgtk(f""" query -i $GRAPH/claims.wikibase-item.augmented.tsv.gz 
         -i p279star 
         -i label 
         -i {vector_output_path} 
         -i $GRAPH/derived.P31x.tsv 
         --match 'item: (n1)-[]->(), 
             P31x: (n1)-[]->(c), 
             p279star: (c)-[]->(class), 
             label: (n1)-[]->(n1_label), 
             label: (class)-[]->(class_label), embeddings: (n1)-[l]->(embedding)'
        --where 'class in ["Q11424", "Q33999", "Q4022", "Q82955"]' 
        --return 'distinct n1, 
                  l.label as label,
                  embedding as node2,
                  kgtk_lqstring_text(n1_label) as `node1;label`, 
                  group_concat(distinct class) as type, 
                  group_concat(distinct kgtk_lqstring_text(class_label)) as `type;label`'
        -o $TEMP/arnold.embeddings.google.projector.tsv
""")

CPU times: user 52 ms, sys: 19.2 ms, total: 71.2 ms
Wall time: 7.68 s


#### Take a peek at the file

In [24]:
kgtk("""head -i $TEMP/arnold.embeddings.google.projector.tsv""")

,node1,label,node2,node1;label,type,type;label
0,Q1000881,graph_embeddings,"-0.032038487,0.082956292,-0.130715191,-0.09142...",Erlau,Q4022,river
1,Q1001872,graph_embeddings,"0.019799879,-0.092170797,-0.136902407,-0.11620...",Buersbach,Q4022,river
2,Q1004531,graph_embeddings,"0.121964820,0.162686124,0.107525878,-0.0407307...",Bullets Over Broadway,Q11424,film
3,Q1009788,graph_embeddings,"0.038406793,-0.380780429,0.375907212,1.0412685...",The Conversation,Q11424,film
4,Q1010099,graph_embeddings,"0.256435424,-0.141215995,0.271997064,-0.359428...",Get Carter,Q11424,film
5,Q1012216,graph_embeddings,"-0.182498246,0.012274691,0.051274516,0.3976519...",Gorillas in the Mist,Q11424,film
6,Q101410,graph_embeddings,"0.504463255,-0.216269359,0.356465191,-0.712683...",François Fillon,Q82955,politician
7,Q101797,graph_embeddings,"-0.479431748,-0.373089701,0.897195935,0.066916...",Winona Ryder,Q33999,actor
8,Q1018487,graph_embeddings,"0.380575299,-0.749318242,0.281814754,0.6557057...",Bye Bye Birdie,Q11424,film
9,Q102124,graph_embeddings,"-0.237421170,-0.299820870,0.535541415,0.043220...",Sigourney Weaver,Q33999,actor


#### Define a function to build the required files for google projector

In [25]:
def build_embedding_projector_metadata(gp_embeddings_path, metadata_path, vectors_path):
    """
    build the vector and metadata files required for google projector
    
    :param gp_embeddings_path: file path which has the embeddings and metadata in kgtk format
    :param metadata_path: output file path for metadata
    :param vectors_path: output file path for vectors
    """
    metadata_file = open(metadata_path, "w")
    metadata_file.write("tag\tqnode\ttype\ttype_label\n")

    vectors_file = open(vectors_path, "w")

    with open(gp_embeddings_path) as qnodes_file:
        next(qnodes_file)
        for line in qnodes_file:
            vals = line.split('\t')
            qnode = vals[0]
            qnode_label = vals[3]
            _type = vals[4] 
            ftype_label = vals[5]
            embeddings = "\t".join(vals[2].strip().split(","))

            if qnode.startswith("Q"):
                metadata_file.write("{}\t{}\t{}\t{}\n".format(qnode_label, qnode, _type, ftype_label.strip()))
                vectors_file.write(embeddings)
                vectors_file.write('\n')

    metadata_file.close()
    vectors_file.close()

In [26]:
build_embedding_projector_metadata(f"{os.environ['TEMP']}/arnold.embeddings.google.projector.tsv",
                                  f"{os.environ['OUT']}/arnold.metadata.{vector_dimension}.tsv",
                                  f"{os.environ['OUT']}/arnold.vectors.{vector_dimension}.tsv")

#### Peek at the metadata file

In [27]:
kgtk(f"""head -i $OUT/arnold.metadata.{vector_dimension}.tsv""")

,tag,qnode,type,type_label
0,Erlau,Q1000881,Q4022,river
1,Buersbach,Q1001872,Q4022,river
2,Bullets Over Broadway,Q1004531,Q11424,film
3,The Conversation,Q1009788,Q11424,film
4,Get Carter,Q1010099,Q11424,film
5,Gorillas in the Mist,Q1012216,Q11424,film
6,François Fillon,Q101410,Q82955,politician
7,Winona Ryder,Q101797,Q33999,actor
8,Bye Bye Birdie,Q1018487,Q11424,film
9,Sigourney Weaver,Q102124,Q33999,actor


#### Peek at the vectors file

In [28]:
!head -2 $OUT/arnold.vectors.$VECTOR_DIMENSION.tsv

-0.032038487	0.082956292	-0.130715191	-0.091424748	0.018285550	0.604146481	0.056206536	-0.261090249	-0.143976569	-0.302393794	-0.268816113	0.079425797	-0.216290280	0.219396368	-0.444404364	0.311044157	0.188877180	-0.091305301	0.270803064	-0.056089807	0.183625966	0.108084373	0.041935548	0.220219851	0.033459932	-0.025080977	0.188042000	-0.129688635	0.997903883	-1.263312697
0.019799879	-0.092170797	-0.136902407	-0.116202936	-0.044151794	0.266715109	0.085795447	-0.505481243	0.064348206	-0.347465873	0.152634412	-0.082999893	0.155982748	0.701899171	-0.402232379	0.467516363	0.576039851	-0.142403096	0.332088053	-0.007845446	0.233441517	0.280902267	0.007143565	0.736638308	-0.100583389	0.432786256	0.329806030	-0.022973346	0.605935633	-1.562803507
